In [1]:
import numpy as np
import pandas as pd
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [18]:
bin_path = 'data/Kitti/sequences/00/velodyne/000000.bin'
kitti = np.fromfile(bin_path, dtype=np.float32).reshape(-1, 4)
kitti = kitti[:, :3]

In [36]:
# bin_path = 'data/Rellis/sequences/00/os1_cloud_node_kitti_bin/000000.bin'
bin_path = 'data/Rellis/sequences/00/vel_cloud_node_kitti_bin/000000.bin'
rellis = np.fromfile(bin_path, dtype=np.float32).reshape(-1, 4)
rellis = rellis[:, :3] # Ignore Reflectivity

In [32]:
kitti.min(), kitti.max(), rellis.min(), rellis.max()

(-78.087395, 77.96733, -58.56, 91.25)

In [39]:
unique_points, counts = np.unique(rellis, axis=0, return_counts=True)
repeat_points = unique_points[counts > 1]
print(repeat_points)

[]


In [2]:
from geotransformer.utils.common import load_pickle

In [4]:
train_pkl = load_pickle('data/Rellis/metadata/train.pkl')
val_pkl = load_pickle('data/Rellis/metadata/val.pkl')
test_pkl = load_pickle('data/Rellis/metadata/test.pkl')

In [5]:
print(len(train_pkl))

889


In [3]:
test_pkl = load_pickle('data/Rellis/metadata/test.pkl')
test_pkl_kitti = load_pickle('data/Kitti/metadata/test.pkl')

In [7]:
val_pkl = load_pickle('data/Rellis/metadata/val.pkl')
val_pkl_kitti = load_pickle('data/Kitti/metadata/val.pkl')

In [5]:
from scipy.spatial.transform import Rotation

def calculate_average_distance(transforms):
    distances = []
    for i in range(len(transforms)-1):
        t1 = transforms[i]['transform']
        t2 = transforms[i+1]['transform']
        r1 = Rotation.from_matrix(t1[:3, :3])
        r2 = Rotation.from_matrix(t2[:3, :3])
        distance = np.linalg.norm(r1.as_rotvec() - r2.as_rotvec())
        distances.append(distance)
    average_distance = sum(distances) / len(distances)
    return average_distance

average_distance_test_pkl = calculate_average_distance(test_pkl)
average_distance_test_pkl_kitti = calculate_average_distance(test_pkl_kitti)

average_distance_test_pkl, average_distance_test_pkl_kitti

(0.16467513796191116, 0.1204392914470209)

In [14]:
val_pkl[10]['transform']

array([[ 0.99877711,  0.01967604, -0.04535084, -6.65557728],
       [-0.02182467,  0.99863786, -0.047389  , -0.06118902],
       [ 0.04435672,  0.04832071,  0.99784677, -0.10037078],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [15]:
val_pkl_kitti[10]['transform']

array([[ 9.99384314e-01,  3.49318770e-02, -3.28247803e-03,
        -9.72660996e+00],
       [-3.49449013e-02,  9.99381329e-01, -3.99638252e-03,
         1.20828686e-01],
       [ 3.14084615e-03,  4.10863526e-03,  9.99986636e-01,
        -1.33577354e-01],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])

In [16]:
test_pkl[0:3]

[{'seq_id': 4,
  'frame0': 25,
  'frame1': 75,
  'p1_transform': array([[ 0.275415  , -0.960795  ,  0.0319266 , -0.130382  ],
         [ 0.961323  ,  0.275198  , -0.0111047 , -0.423012  ],
         [ 0.00188321,  0.0337502 ,  0.999429  ,  0.167099  ],
         [ 0.        ,  0.        ,  0.        ,  1.        ]]),
  'p2_transform': array([[ 0.274723  , -0.960998  ,  0.0317688 , -0.14286   ],
         [ 0.961522  ,  0.274515  , -0.010813  , -0.440052  ],
         [ 0.00167028,  0.033517  ,  0.999437  ,  0.162578  ],
         [ 0.        ,  0.        ,  0.        ,  1.        ]]),
  'transform': array([[ 1.00000031e+00, -7.12932818e-04,  2.36972677e-04,
          -1.98261112e-02],
         [ 7.12448083e-04,  9.99999211e-01,  2.32158610e-04,
           7.14684854e-03],
         [-2.37111249e-04, -2.31963216e-04,  9.99999718e-01,
          -4.72757028e-03],
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
           1.00000000e+00]]),
  'pcd0': 'downsampled/04/000025.npy',
  '

In [7]:
gt_transforms = pd.read_csv('data/Rellis/sequences/04/poses.txt', header=None, sep=' ')

In [21]:
test_pkl[0]['p1_transform'] == np.concatenate([gt_transforms.iloc[25].to_numpy().reshape(3,4), np.array([[0,0,0,1]])], axis=0)
test_pkl[0]['p2_transform'] == np.concatenate([gt_transforms.iloc[75].to_numpy().reshape(3,4), np.array([[0,0,0,1]])], axis=0)

array([[ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True]])

In [ ]:
t1 = test_pkl[0]['p1_transform']
t2 = test_pkl[0]['p2_transform']
t3 = test_pkl[1]['p2_transform']

T = np.linalg.inv(t1) @ t2
T2 = np.linalg.inv(t2) @ t3

t2_recon = t1 @ T
t3_recon = t2_recon @ T2

t2, t2_recon, t3, t3_recon

In [8]:
gt = test_pkl[0]['p1_transform']

for transform in test_pkl:
    gt = gt @ transform['transform']

In [17]:
gt.astype(np.float32)==test_pkl[-1]['p2_transform'].astype(np.float32), gt.astype(np.float32) == np.concatenate([gt_transforms.iloc[2025].to_numpy().reshape(3,4), np.array([[0,0,0,1]])], axis=0).astype(np.float32)

(array([[ True,  True,  True,  True],
        [ True,  True,  True,  True],
        [ True,  True,  True,  True],
        [ True,  True,  True,  True]]),
 array([[ True,  True,  True,  True],
        [ True,  True,  True,  True],
        [ True,  True,  True,  True],
        [ True,  True,  True,  True]]))

In [15]:
np.concatenate([gt_transforms.iloc[2025].to_numpy().reshape(3,4), np.array([[0,0,0,1]])], axis=0)

array([[ 5.39052e-01,  8.42228e-01,  8.61829e-03,  5.23655e+01],
       [-8.40897e-01,  5.37559e-01,  6.26386e-02, -2.06677e+02],
       [ 4.81232e-02, -4.10125e-02,  9.97999e-01,  1.10181e+00],
       [ 0.00000e+00,  0.00000e+00,  0.00000e+00,  1.00000e+00]])

In [9]:
test_pkl[-1]

{'seq_id': 4,
 'frame0': 1975,
 'frame1': 2025,
 'p1_transform': array([[ 5.45210e-01,  8.38285e-01,  4.80935e-03,  5.23804e+01],
        [-8.36425e-01,  5.43599e-01,  6.99586e-02, -2.06662e+02],
        [ 5.60309e-02, -4.21648e-02,  9.97538e-01,  1.10878e+00],
        [ 0.00000e+00,  0.00000e+00,  0.00000e+00,  1.00000e+00]]),
 'p2_transform': array([[ 5.39052e-01,  8.42228e-01,  8.61829e-03,  5.23655e+01],
        [-8.40897e-01,  5.37559e-01,  6.26386e-02, -2.06677e+02],
        [ 4.81232e-02, -4.10125e-02,  9.97999e-01,  1.10181e+00],
        [ 0.00000e+00,  0.00000e+00,  0.00000e+00,  1.00000e+00]]),
 'transform': array([[ 9.99940006e-01,  7.26633310e-03,  8.22513111e-03,
          4.03218981e-03],
        [-7.25971063e-03,  9.99973440e-01, -8.05597899e-04,
         -2.03505491e-02],
        [-8.23070675e-03,  7.45877279e-04,  9.99966087e-01,
         -8.07388196e-03],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          1.00000000e+00]]),
 'pcd0': 'downsampled/04/